In [1]:
import pandas as pd
from time import time
import numpy as np

In [2]:
start = time()
pkls =pd.read_feather(r'\\fatherserverdw\q\research\images\skin_aging\wsi\hovernet_out\df\2d_skin_hovernet_inroi.ftr') #42sec local # 90sec network
print(round(time()-start),'seconds')

60 seconds


In [6]:
# pkls=pkls[pkls['inroi']>0].reset_index(drop=True)
# pkls.to_feather(r'\\fatherserverdw\q\research\images\skin_aging\wsi\hovernet_out\df\2d_skin_hovernet_inroi.ftr')

In [6]:
pkls['inroi'].unique(),pkls['type'].unique()

(array([1, 3, 4, 2, 5, 6], dtype=int64),
 array([10,  9,  8, 12,  2,  1,  7,  5,  4,  6, 11,  3], dtype=int64))

In [3]:
agelut = pd.read_excel(r'\\fatherserverdw\Q\research\source code\skin_aging_matlab\skin_samples.xlsx', sheet_name='imlist')

In [4]:
len(pkls)

7856729

In [5]:
pkls.head()

,bbox,centroid,contour,type,inroi,Area,Perimeter,Circularity,MA,ma,orientation,AspectRatio,Sol,Extent,EquiDia,imID,dist2nearest,oriA,oriB,local_align
0,"[[39028, 23927], [39050, 23938]]","[40315.561643835616, 22653.72602739726]","[[40316, 22644], [40316, 22646], [40315, 22647...",10,1,121.5,54.041630,0.522792,7.948583,21.956139,18.977255,0.362021,0.886861,0.502066,12.437789,8,33.837849,27.686026,39.083927,0.352779
1,"[[39049, 23898], [39066, 23912]]","[40287.50326797386, 22672.287581699347]","[[40285, 22665], [40285, 22666], [40283, 22668...",10,1,131.5,48.041630,0.715979,11.356377,17.369274,27.686026,0.653820,0.929329,0.552521,12.939513,8,33.837849,18.977255,39.083927,0.352779
2,"[[39052, 24088], [39066, 24102]]","[40479.20183486238, 22674.963302752294]","[[40481, 22668], [40477, 22672], [40477, 22673...",10,1,90.5,42.041630,0.643427,7.466627,17.245905,46.967815,0.432951,0.932990,0.461735,10.734439,8,52.000000,60.451813,84.023666,0.293905
3,"[[39069, 23255], [39082, 23267]]","[39644.69158878505, 22691.168224299065]","[[39645, 22685], [39643, 22687], [39643, 22688...",10,1,89.5,39.213203,0.731422,8.626031,14.283838,36.164223,0.603901,0.937173,0.573718,10.674968,8,72.622311,58.224380,104.293274,0.524896
4,"[[39076, 23171], [39086, 23187]]","[39562.37096774193, 22696.935483870966]","[[39559, 22692], [39558, 22693], [39557, 22693...",10,1,105.0,40.970562,0.786059,9.389393,15.466069,104.293274,0.607096,0.967742,0.656250,11.562446,8,35.355339,76.607521,59.920910,0.279197


In [8]:
pkls.loc[pkls['type']==12]=10 #replace cells in white to ecm since its just gap
pkls.loc[pkls['type']==1]=2 # all corneum cells are spinosum
# pkls.loc[pkls['type']==3]=4 # all shaft cells are follicle

In [13]:
pkl = pkls[pkls['imID']==8]
pkl = pkl.drop(columns=['bbox', 'centroid','contour'])
pk = pkl[pkl['inroi']==1]
p = pk[pk['type']==1]
pkmean = p.mean(axis=0)
pkmean['Ncell'] = len(p)
pkmean['imID'] = 8
pkmean['Age'] = agelut['Age'][agelut['redcap']==8].values[0]
pkmean

type             NaN
inroi            NaN
Area             NaN
Perimeter        NaN
Circularity      NaN
MA               NaN
ma               NaN
orientation      NaN
AspectRatio      NaN
Sol              NaN
Extent           NaN
EquiDia          NaN
imID             8.0
dist2nearest     NaN
oriA             NaN
oriB             NaN
local_align      NaN
Ncell            0.0
Age             11.6
dtype: float64

In [ ]:
dl = Image.open(r'\\fatherserverdw\Q\research\images\skin_aging\annotation\roi\tif\8_tissue_binary.tif')
            dlareas = dl.histogram()[1:13]

In [14]:
%%time
avgs = []
stds = []
for imid in pkls['imID'].unique():
    pkl = pkls[pkls['imID']==imid]
    pkl = pkl.drop(columns=['bbox', 'centroid','contour'])
    for secid in pkl['inroi'].unique():
        pk = pkl[pkl['inroi']==secid]
        for type in range(2,12):
            p = pk[pk['type']==type]
            pkmean = p.mean(axis=0)
            pkmean['Ncell'] = len(p)
            pkmean['imID'] = imid
            pkmean['Age'] = agelut['Age'][agelut['redcap']==imid].values[0]
            pkstd = p.std(axis=0)
            pkstd['imID'] = imid
            pkstd['type'] = type
            pkstd['inroi'] = secid
            pkstd['Age'] = agelut['Age'][agelut['redcap']==imid].values[0]
            stds.append(pkstd.replace(np.nan,0))
            avgs.append(pkmean.replace(np.nan, 0))

CPU times: total: 16 s
Wall time: 12.3 s


In [15]:
avgdf = pd.DataFrame(avgs)
stddf = pd.DataFrame(stds)

In [16]:
avgdf.to_csv('avgdf.csv')
stddf.to_csv('stddf.csv')

In [ ]:
avg_lvl_corr=[]
for type in avgdf['type'].unique():
    adf = avgdf[avgdf['type']==type]
    corr = adf.corrwith(adf['age'], method='kendall')
    avg_lvl_corr.append(corr)